# Zindi - Challenge

https://zindi.africa/competitions/predict-the-global-spread-of-covid-19

April 9, 2020

In [226]:
import pandas as pd
import seaborn as sns
import matplotlib as plt 
import requests
import lxml.html as lh

## Import Data

In [254]:
#Read train 
deaths = pd.read_csv('data/deaths.csv')
cases = pd.read_csv('data/cases.csv')
deaths.drop(['Province/State'], axis=True)
deaths.shape

(274, 84)

In [255]:
deaths.head(2)

,Province/State,Country/Region,Territory,Population,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,...,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20
0,NaN,Afghanistan,Afghanistan,26023100.0,33.0000,65.0000,0,0,0,0,...,4,4,4,6,6,7,7,11,14,14
1,NaN,Albania,Albania,2895947.0,41.1533,20.1683,0,0,0,0,...,11,15,15,16,17,20,20,21,22,22


Compare the 1st 3 columns.

In [256]:
#no missing values
sum(deaths['Country/Region'].isnull() == False)

274

In [257]:
l1=deaths['Country/Region'].unique()
l2=deaths['Territory'].unique()
len([value for value in l1 if value in l2] )

169

In [258]:
len([x for x in l2 if x not in l1])
[x for x in l2 if x not in l1]

['Bahamas (the)',
 'Bolivia (Plurinational State of)',
 'Brunei Darussalam',
 'Central African Republic (the)',
 'Congo (the)',
 'Democratic Republic of the Congo (the)',
 "Côte d'Ivoire",
 'Faroe Islands',
 'Greenland',
 'Dominican Republic (the)',
 'French Guiana',
 'French Polynesia',
 'Guadeloupe',
 'Mayotte',
 'New Caledonia',
 'Reunion',
 'Saint Barthelemy',
 'St Martin',
 'Gambia (the)',
 'Iran (Islamic Republic of)',
 "Democratic People's Republic of Korea (the)",
 'Republic of Moldova (the)',
 'Aruba',
 'Curacao',
 'Netherlands (the)',
 'Niger (the)',
 'Philippines (the)',
 'Russian Federation (the)',
 'Taiwan',
 'United Republic of Tanzania (the)',
 'United Arab Emirates (the)',
 'Bermuda',
 'Cayman Islands',
 'United Kingdom of Great Britain and Northern Ireland (the)',
 'Montserrat',
 'United States of America (the)',
 'Venezuela (Bolivarian Republic of)',
 'Viet Nam',
 'Syrian Arab Republic (the)',
 "Lao People's Democratic Republic (the)"]

In [259]:
len([x for x in l1 if x not in l2])
[x for x in l1 if x not in l2]

['Bahamas',
 'Bolivia',
 'Brunei',
 'Central African Republic',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 "Cote d'Ivoire",
 'Dominican Republic',
 'Gambia',
 'Iran',
 'Korea, South',
 'Moldova',
 'Netherlands',
 'Niger',
 'Philippines',
 'Russia',
 'Sudan',
 'Taiwan*',
 'Tanzania',
 'United Arab Emirates',
 'United Kingdom',
 'US',
 'Venezuela',
 'Vietnam',
 'Syria',
 'Laos']

In [260]:
len(deaths['Territory'].unique())

209

In [261]:
len(deaths['Province/State'].unique())

84

In [262]:
deaths[deaths['Province/State'].isnull() == False]

,Province/State,Country/Region,Territory,Population,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,...,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20
8,Australian Capital Territory,Australia,Australia,2.369690e+07,-35.4735,149.0124,0,0,0,0,...,1,1,1,1,1,2,2,2,2,2
9,New South Wales,Australia,Australia,2.369690e+07,-33.8688,151.2093,0,0,0,0,...,8,8,9,10,12,12,16,18,21,21
10,Northern Territory,Australia,Australia,2.369690e+07,-12.4634,130.8456,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,Queensland,Australia,Australia,2.369690e+07,-28.0167,153.4000,0,0,0,0,...,2,2,2,4,4,4,4,4,4,4
12,South Australia,Australia,Australia,2.369690e+07,-34.9285,138.6007,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
13,Tasmania,Australia,Australia,2.369690e+07,-41.4545,145.9707,0,0,0,0,...,0,1,2,2,2,2,2,2,2,3
14,Victoria,Australia,Australia,2.369690e+07,-37.8136,144.9631,0,0,0,0,...,4,4,4,5,7,8,8,10,11,12
15,Western Australia,Australia,Australia,2.369690e+07,-31.9505,115.8605,0,0,0,0,...,2,2,2,2,2,2,3,4,4,6
35,Alberta,Canada,Canada,3.554042e+07,53.9333,-116.5765,0,0,0,0,...,3,8,9,13,13,18,20,23,24,26
36,British Columbia,Canada,Canada,3.554042e+07,49.2827,-123.1207,0,0,0,0,...,19,24,24,31,31,38,38,38,39,43


In [263]:
len(deaths['Province/State'].unique())

84

We will use 'Country/Region'. 


In [264]:
country = deaths['Country/Region']

In [265]:
deaths.head(2)

,Province/State,Country/Region,Territory,Population,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,...,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20
0,NaN,Afghanistan,Afghanistan,26023100.0,33.0000,65.0000,0,0,0,0,...,4,4,4,6,6,7,7,11,14,14
1,NaN,Albania,Albania,2895947.0,41.1533,20.1683,0,0,0,0,...,11,15,15,16,17,20,20,21,22,22


In [266]:
deaths[deaths.Population.isnull()]

,Province/State,Country/Region,Territory,Population,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,...,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20
3,NaN,Andorra,Andorra,NaN,42.506300,1.521800,0,0,0,0,...,8,12,14,15,16,17,18,21,22,23
18,NaN,Bahamas,Bahamas (the),NaN,25.034300,-77.396300,0,0,0,0,...,0,0,1,1,1,4,4,5,6,7
26,NaN,Bolivia,Bolivia (Plurinational State of),NaN,-16.290200,-63.588700,0,0,0,0,...,4,6,7,8,9,10,10,11,14,15
29,NaN,Brunei,Brunei Darussalam,NaN,4.535300,114.727700,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
32,NaN,Cabo Verde,Cabo Verde,NaN,16.538800,-23.041800,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
46,NaN,Central African Republic,Central African Republic (the),NaN,6.611100,20.939400,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
83,NaN,Congo (Brazzaville),Congo (the),NaN,-4.038300,21.758700,0,0,0,0,...,0,0,0,2,2,2,5,5,5,5
84,NaN,Congo (Kinshasa),Democratic Republic of the Congo (the),NaN,-4.038300,21.758700,0,0,0,0,...,8,8,9,13,13,18,18,18,18,18
86,NaN,Cote d'Ivoire,Côte d'Ivoire,NaN,7.540000,-5.547100,0,0,0,0,...,1,1,1,1,1,1,3,3,3,3
90,NaN,Czechia,Czechia,NaN,49.817500,15.473000,0,0,0,0,...,23,31,39,44,53,59,67,78,88,99


In [267]:
## Fill in NaN values for Population
index = deaths[deaths.Population.isnull()].index
# updated in 2017
deaths.at[index[0], 'Population'] = 76965 
deaths.at[index[1], 'Population'] = 395361
deaths.at[index[2], 'Population'] = 11.05e+06
deaths.at[index[3], 'Population'] = 428697
deaths.at[index[4], 'Population'] = 546388 
deaths.at[index[5], 'Population'] = 4.659e+06
deaths.at[index[6], 'Population'] = 5.261e+06
deaths.at[index[7], 'Population'] = 81.34e+06
deaths.at[index[8], 'Population'] = 24.29e+06 # Cote d'Ivoire
deaths.at[index[10], 'Population'] = 10.77e+06
deaths.at[index[11], 'Population'] = 1.367e+06
deaths.at[index[12], 'Population'] = 2.101e+06
deaths.at[index[13], 'Population'] = 81.16e+06
deaths.at[index[14], 'Population'] = 51.47e+06
deaths.at[index[15], 'Population'] = 3.55e+06
deaths.at[index[17], 'Population'] = 105264
deaths.at[index[18], 'Population'] = 161014
deaths.at[index[19], 'Population'] = 41109 
deaths.at[index[21], 'Population'] = 21.48e+06
deaths.at[index[23], 'Population'] = 104.9e+06
deaths.at[index[24], 'Population'] = 144.5e+06
deaths.at[index[25], 'Population'] = 7.022e+06
deaths.at[index[26], 'Population'] = 12.58e+06
deaths.at[index[28], 'Population'] = 57.31e+06
deaths.at[index[29], 'Population'] = 9.4e+06
deaths.at[index[30], 'Population'] = 65441
deaths.at[index[31], 'Population'] = 61559
deaths.at[index[32], 'Population'] = 170499
deaths.at[index[33], 'Population'] = 34571
deaths.at[index[34], 'Population'] = 84287
deaths.at[index[35], 'Population'] = 5900 #2008
deaths.at[index[38], 'Population'] = 31.98e+06
deaths.at[index[39], 'Population'] = 95.54e+06
deaths.at[index[40], 'Population'] = 18.27e+06
deaths.at[index[41], 'Population'] = 1.296e+06
deaths.at[index[42], 'Population'] = 6.858e+06
deaths.at[index[43], 'Population'] = 15094
deaths.at[index[44], 'Population'] = 31196
deaths.at[index[45], 'Population'] = 35446
deaths.at[index[46], 'Population'] = 25157
deaths.at[index[47], 'Population'] = 2840
deaths.at[index[48], 'Population'] = 204327
deaths.at[index[49], 'Population'] = 813912
deaths.at[index[50], 'Population'] = 53127
deaths.at[index[51], 'Population'] = 105544
deaths.at[index[52], 'Population'] = 53.37e+06
deaths.at[index[53], 'Population'] = 51.47e+06
deaths.at[index[54], 'Population'] = 40.53e+06

#2018 
deaths.at[index[27], 'Population'] = 23.78e+06
deaths.at[index[37], 'Population'] = 327.2e+06
#2019
deaths.at[index[9], 'Population'] = 10.65e+06
deaths.at[index[20], 'Population'] = 17.28e+06
deaths.at[index[22], 'Population'] = 2.077e+06
deaths.at[index[36], 'Population'] = 6.65e+06

#2020
deaths.at[index[16], 'Population'] = 631219





In [268]:
deaths[deaths['Country/Region'] =='Andorra']

,Province/State,Country/Region,Territory,Population,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,...,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20
3,NaN,Andorra,Andorra,76965.0,42.5063,1.5218,0,0,0,0,...,8,12,14,15,16,17,18,21,22,23


No more NaN values for population. 

Add a code country column.

In [269]:
def web_scraping_data(link):
    #Create a handle, page, to handle the contents of the website
    page = requests.get(link)
    doc = lh.fromstring(page.content)
    tr_elements = doc.xpath('//tr')

    tr_elements = doc.xpath('//tr')
    col=[]
    i=0
    for t in tr_elements[0]:
        i+=1
        name=t.text_content()
        #print ('%d:"%s"'%(i,name))
        col.append((name,[]))

    for j in range(1,len(tr_elements)):
        T=tr_elements[j]
        i=0

        for t in T.iterchildren():
            data=t.text_content() 
            if i>0:
                try:
                    data=int(data)
                except:
                    pass

            col[i][1].append(data)
            i+=1

    Dict={title:column for (title,column) in col}
    df=pd.DataFrame(Dict)
    
    return df

In [270]:
code_country = web_scraping_data("https://www.iban.com/country-codes")
df = deaths.merge(code_country, how='left', left_on='Country/Region', right_on='Country')

In [288]:
df[df['Alpha-3 code'].isnull()]

,Province/State,Country/Region,Territory,Population,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,...,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,Country,Alpha-2 code,Alpha-3 code,Numeric


In [287]:
## Fill in NaN values for alpha 3 code 
index = df[df['Alpha-3 code'].isnull()].index
variable = 'Alpha-3 code'
df.at[index[0], variable] = 'BHS'
df.at[index[1], variable] = 'BOL'
df.at[index[2], variable] = 'BRN'
df.at[index[3], variable] = 'CAF'
df.at[index[4], variable] = 'COG'
df.at[index[5], variable] = 'COD'
df.at[index[6], variable] = 'CIV'
df.at[index[7], variable] = 'DOM'
df.at[index[8], variable] = 'GMB'
df.at[index[9],variable] = 'IRN'
df.at[index[10],variable] = 'PRK'
df.at[index[11], variable] = 'MDA'
df.at[index[12], variable] = 'ABW'
df.at[index[13], variable] = 'CUW'
df.at[index[14],variable] = 'SXM'
df.at[index[15],variable] = 'NLD'
df.at[index[16],variable] = 'NER'
df.at[index[17], variable] = 'MKD'
df.at[index[18], variable] = 'PHL'
df.at[index[19], variable] = 'RUS'
df.at[index[20], variable] = 'SSD'
df.at[index[21], variable] = 'TWN'
df.at[index[22],variable] = 'TZA'
df.at[index[23],variable] = 'ARE'
df.at[index[24],variable] = 'BMU'
df.at[index[25],variable] = 'CYM'
df.at[index[26],variable] = 'GBR' #by default
df.at[index[27],variable] = 'GIB'
df.at[index[28],variable] = 'IMN'
df.at[index[29],variable] = 'MSR'
df.at[index[30],variable] = 'GBR'
df.at[index[31],variable] = 'USA'
df.at[index[32], variable] = 'VEN'
df.at[index[33],variable] = 'VNM'
df.at[index[34],variable] = 'SYR'
df.at[index[35], variable] = 'LAO'
df.at[index[36], variable] = 'AIA'
df.at[index[37], variable] = 'VGB'
df.at[index[38], variable] = 'TCA'
df.at[index[39],variable] = 'BES'
df.at[index[40], variable] = 'FLK'
df.at[index[41], variable] = 'KOR'


We will use alpha 3 code to join with other tables.

## Add variables


### Temperature 

Source 
- https://datacatalog.worldbank.org/dataset/climate-change-knowledge-portal-historical-data

In [290]:
temp = pd.read_csv('variable/temp.csv')

In [291]:
temp.shape

(178, 14)

In [192]:
temperature = pd.merge(left=temp, right=df, left_on='ISO_3DIGIT', right_on='Alpha-3 code')

In [193]:
temperature = temperature.drop(['Numeric','Alpha-2 code'], axis=True)

In [188]:
temperature.drop('Country', axis =1, inplace=True)

In [189]:
death1 = pd.merge(left=deaths, right=temperature, left_on='Territory', right_on='Country')

KeyError: 'Country'

In [179]:
death1.shape

(229, 100)

I am loosing 45 countries which are 

In [180]:
l1 = list(death1.Country.unique())
l2 = list(deaths['Country/Region'].unique())
len([x for x in l2 if x not in l1])

53

In [181]:
[x for x in l2 if x not in l1]

['Andorra',
 'Antigua and Barbuda',
 'Bahamas',
 'Bahrain',
 'Barbados',
 'Bolivia',
 'Brunei',
 'Cabo Verde',
 'Central African Republic',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 "Cote d'Ivoire",
 'Dominican Republic',
 'Gambia',
 'Iran',
 'Korea, South',
 'Liechtenstein',
 'Maldives',
 'Malta',
 'Moldova',
 'Monaco',
 'Netherlands',
 'Niger',
 'North Macedonia',
 'Philippines',
 'Russia',
 'Saint Lucia',
 'San Marino',
 'Seychelles',
 'Singapore',
 'Sudan',
 'Taiwan*',
 'Tanzania',
 'United Arab Emirates',
 'United Kingdom',
 'US',
 'Venezuela',
 'Vietnam',
 'Dominica',
 'Grenada',
 'Syria',
 'Laos',
 'Saint Kitts and Nevis',
 'Sao Tome and Principe',
 'Kiribati',
 'Marshall Islands (the)',
 'Micronesia (Federated States of)',
 'Nauru',
 'Palau',
 'Republic of Korea (the)',
 'Samoa',
 'Tonga',
 'Tuvalu']

In [186]:
death1[death1.Population.isnull()]

,Province/State,Country/Region,Territory,Population,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,...,Jun_Temp,July_Temp,Aug_Temp,Sept_temp,Oct_temp,Nov_Temp,Dec_temp,Annual_temp,Country,Alpha-3 code
16,NaN,Bahamas,Bahamas (the),NaN,25.034300,-77.396300,0,0,0,0,...,27.23,27.95,27.99,27.68,26.45,24.44,22.52,25.06,Bahamas (the),BHS
22,NaN,Bolivia,Bolivia (Plurinational State of),NaN,-16.290200,-63.588700,0,0,0,0,...,17.65,17.77,19.43,21.33,22.51,22.91,22.78,20.98,Bolivia (Plurinational State of),BOL
25,NaN,Brunei,Brunei Darussalam,NaN,4.535300,114.727700,0,0,0,0,...,26.26,26.04,26.08,26.05,25.92,25.48,25.69,25.93,Brunei Darussalam,BRN
45,NaN,Central African Republic,Central African Republic (the),NaN,6.611100,20.939400,0,0,0,0,...,24.65,23.77,23.71,23.84,24.31,24.25,23.65,24.84,Central African Republic (the),CAF
82,NaN,Congo (Brazzaville),Congo (the),NaN,-4.038300,21.758700,0,0,0,0,...,23.82,23.03,23.34,24.05,24.54,24.52,24.47,24.52,Congo (the),COG
84,NaN,Cote d'Ivoire,Côte d'Ivoire,NaN,7.540000,-5.547100,0,0,0,0,...,25.88,24.87,24.74,25.27,25.99,26.32,25.43,26.30,Côte d'Ivoire,CIV
88,NaN,Czechia,Czechia,NaN,49.817500,15.473000,0,0,0,0,...,15.41,17.11,16.90,13.49,8.59,2.82,-1.06,7.67,Czechia,CZE
91,NaN,Dominican Republic,Dominican Republic (the),NaN,18.735700,-70.162700,0,0,0,0,...,25.01,25.21,25.64,25.35,24.73,23.75,22.52,23.93,Dominican Republic (the),DOM
98,NaN,Eswatini,Eswatini,NaN,-26.522500,31.465900,0,0,0,0,...,15.72,15.91,17.62,19.77,20.62,21.78,23.26,20.41,Eswatini,SWZ
109,NaN,Gambia,Gambia (the),NaN,13.443200,-15.310100,0,0,0,0,...,28.55,27.36,26.73,26.73,27.59,26.52,23.93,26.77,Gambia (the),GMB


### Others epidemy 

SARS-CoV

China Canada, Hong Kong Special Administrative Region of China, Chinese Taipei, Singapore, and Hanoi in Viet Nam.

Ebola 

The Democratic Republic of the Congo
Gabon
Guinea
Italy
Ivory Coast
Liberia
Mali
Nigeria
Philippines
Russia
Senegal
Sierra Leone
South Africa
South Sudan
Spain
Uganda
United Kingdom
United States
What causes Ebol

Mers - https://www.kaggle.com/imdevskp/mers-outbreak-dataset-20122019

Ebola - https://www.kaggle.com/imdevskp/ebola-outbreak-20142016-complete-dataset

Sars - https://www.kaggle.com/imdevskp/sars-outbreak-2003-complete-dataset

### Age / density / wealth /sex 

http://datatopics.worldbank.org/universal-health-coverage/coronavirus/

See relevant indicators

https://datacatalog.worldbank.org/search?search_api_views_fulltext_op=AND&f%5B0%5D=field_collection_field%3A2026&sort_by=field_wbddh_modified_date

https://data.worldbank.org/indicator/en.pop.dnst

In [13]:
def wordbankdata(variable, tab):
    data = pd.read_csv('variable/'+variable+'.csv')
    data = data[['Country Code', '2018']]
    death = pd.merge(left=data, right=tab, left_on='Country Code', right_on='Alpha-3 code')
    death[variable] = death['2018']
    death.drop(['2018'], axis=True, inplace=True)
    return(death)

variable = '%+65'

In [14]:
death2 = wordbankdata(variable, death1)

In [15]:
l1 = list(death1.Country)
l2 = list(death2.Country)
[x for x in l1 if x not in l2]

['French Guiana', 'Guadeloupe']

In [16]:
variable = '15-64pop'
death2 = wordbankdata(variable, death2)
death2 = wordbankdata('density', death2)
death2 = wordbankdata( 'sex', death2)

In [17]:
rich = pd.read_csv('variable/income.csv')
rich = rich[['Country Code', 'IncomeGroup']]

In [18]:
death2 = pd.merge(left=rich, right=death2, left_on='Country Code', right_on='Alpha-3 code')

In [19]:
death2.drop(['Alpha-3 code','Country Code_y', 'Country Code_x','Country Code_y' ,'Country'], axis=1, inplace=True)


In [20]:
death2.head()

,Country Code,IncomeGroup,Province/State,Country/Region,Territory,Population,Lat,Long,1/22/20,1/23/20,...,Aug_Temp,Sept_temp,Oct_temp,Nov_Temp,Dec_temp,Annual_temp,%+65,15-64pop,density,sex
0,AFG,Low income,NaN,Afghanistan,Afghanistan,26023100.0,33.0000,65.0000,0,0,...,23.77,19.03,12.99,7.00,2.43,12.92,2.584927,54.324898,56.937760,48.635847
1,AGO,Lower middle income,NaN,Angola,Angola,24383301.0,-11.2027,17.8739,0,0,...,19.90,22.19,23.18,22.79,22.61,21.51,2.216374,50.974702,24.713052,50.530463
2,ALB,Upper middle income,NaN,Albania,Albania,2895947.0,41.1533,20.1683,0,0,...,20.48,17.16,12.27,7.58,3.65,11.27,13.744736,68.582390,104.612263,49.063095
3,ARE,High income,NaN,United Arab Emirates,United Arab Emirates (the),NaN,24.0000,54.0000,0,0,...,33.55,31.74,28.34,24.06,20.28,26.83,1.085001,84.311490,135.609110,30.636688
4,ARG,Upper middle income,NaN,Argentina,Argentina,42669500.0,-38.4161,-63.6167,0,0,...,9.02,11.53,14.67,17.54,19.83,14.22,11.117789,64.121277,16.258510,51.237348


In [21]:
death2.shape

(227, 104)

In [22]:
deaths.shape

(274, 84)

## Tourism

In [43]:
tourism = pd.read_csv('variable/tourism.csv')

tourism.count() / len(tourism)
Lets take 2011.

In [50]:
tourism = tourism[['2011','Country Code']]

## Economical factors

https://www.kaggle.com/lewisduncan93/the-economic-freedom-index

## Immunization factor

https://www.kaggle.com/lsind18/who-immunization-coverage

who-immunization-coverage folder

## Split by continent

In [23]:
continent = pd.read_csv('variable/locations.csv')

In [24]:
continent.head()

,countriesAndTerritories,location,continent,population_year,population
0,Afghanistan,Afghanistan,Asia,2020.0,38928341.0
1,Albania,Albania,Europe,2020.0,2877800.0
2,Algeria,Algeria,Africa,2020.0,43851043.0
3,Andorra,Andorra,Europe,2020.0,77265.0
4,Angola,Angola,Africa,2020.0,32866268.0


In [25]:
continent.shape

(207, 5)

In [26]:
continent.drop(['population_year','population','countriesAndTerritories'], axis=1, inplace=True)

In [27]:
death2 = pd.merge(left=continent, right=death1, left_on='location', right_on='Country/Region')

In [28]:
death2.shape

(216, 102)

### Infected cases March 1 / March 15 / March 30

We take the number of cases every 15 days since the beginning of March. 

In [ ]:
cases = pd.read_csv('data/cases.csv')

In [ ]:
cases[['2/15/20', '3/1/20', '3/30/20', '3/30/20', 'Country/Region']]

## Commorbidity / Risk Factors

### Obesity 

https://www.kaggle.com/arttua/who-obesity-by-country-2016

In [30]:
obesity = pd.read_csv('variable/obesity.csv')
obesity['Country'] = obesity['Unnamed: 0']
obesity.drop('Unnamed: 0', axis=1, inplace=True)

In [31]:
death3 = pd.merge(left=obesity, right=death2, left_on='Country', right_on='Country/Region')
death3.shape

(200, 106)

In [32]:
death2.shape

(216, 102)

### Pollution ?
https://www.kaggle.com/kweinmeister/pm25-global-air-pollution-20102017

## Medical care

### Hospital beds

https://www.kaggle.com/hamzael1/hospital-beds-by-country

In [33]:
beds = pd.read_csv('variable/beds.csv')

In [34]:
beds.count() / len(beds)

Country Name      1.000000
Country Code      1.000000
Indicator Name    1.000000
Indicator Code    1.000000
1960              0.613636
1961              0.007576
1962              0.007576
1963              0.007576
1964              0.007576
1965              0.030303
1966              0.018939
1967              0.018939
1968              0.018939
1969              0.018939
1970              0.662879
1971              0.030303
1972              0.034091
1973              0.034091
1974              0.041667
1975              0.340909
1976              0.056818
1977              0.041667
1978              0.045455
1979              0.045455
1980              0.492424
1981              0.280303
1982              0.113636
1983              0.098485
1984              0.125000
1985              0.378788
                    ...   
1991              0.363636
1992              0.318182
1993              0.412879
1994              0.318182
1995              0.318182
1996              0.443182
1

In [35]:
beds= beds[['Country Code', '2011']]

MERGE !

### Health Systems - WHO 


In [36]:
hs = pd.read_csv('variable/Health_systems.csv')

In [37]:
hs.shape

(210, 14)

In [39]:
hs.head()

,Country_Region,Province_State,World_Bank_Name,Health_exp_pct_GDP_2016,Health_exp_public_pct_2016,Health_exp_out_of_pocket_pct_2016,Health_exp_per_capita_USD_2016,per_capita_exp_PPP_2016,External_health_exp_pct_2016,Physicians_per_1000_2009-18,Nurse_midwife_per_1000_2009-18,Specialist_surgical_per_1000_2008-18,Completeness_of_birth_reg_2009-18,Completeness_of_death_reg_2008-16
0,Afghanistan,NaN,Afghanistan,10.2,5.1,77.4,57.2,162.8,17.5,0.3,0.3,0.0,42.3,NaN
1,Albania,NaN,Albania,6.7,41.4,58.0,271.5,759.7,0.7,1.2,3.6,11.6,98.4,53.0
2,Algeria,NaN,Algeria,6.6,67.7,30.9,260.4,998.2,0.0,1.8,2.2,12.1,100.0,NaN
3,Andorra,NaN,Andorra,10.4,49.1,41.7,3834.7,4978.7,NaN,3.3,4.0,83.1,100.0,80.0
4,Angola,NaN,Angola,2.9,44.1,35.2,95.2,185.8,3.6,0.2,1.3,NaN,25.0,NaN


### Test

https://www.kaggle.com/skylord/covid19-tests-conducted-by-country

In [41]:
test = pd.read_csv('variable/test/Tests_conducted_09April2020.csv')

import all files in the folder test !

## Vaccine - clinical test
https://www.kaggle.com/panahi/covid-19-international-clinical-trials#ClinicalTrials.gov_COVID_19.csv

https://www.kaggle.com/zohrarezgui/coronavirus-clinical-trials-dataset

## Non-pharmaceutical interventions



### Lockdown 

https://www.kaggle.com/jcyzag/covid19-lockdown-dates-by-country

lockdown folder !

In [ ]:
sum(deaths.Territory == 'France')

In [ ]:
deaths[deaths.Territory == 'France']